This example assumes that you have an [Ollama](https://github.com/ollama/ollama) model running (with `ollama serve`) on some host that is reachable (whether a `localhost` of your computer or some other host with a public IP).
As a predictor, we use [txai_omics_3](https://github.com/SermanVS/txai_omics_3).

In [1]:
from llamba.chatmodels.ollama import OllamaModel

In [2]:
chatbot = OllamaModel(url="http://127.0.0.1:11434/", endpoint="api/generate", model="llama3", num_threads=1, check_connection_timeout=15, request_timeout=15)
connection = chatbot.check_connection()
print(connection)

True


### Get a bioage model

In [3]:
from pathlib import Path
import os
import shap
import numpy as np
import pickle
import torch
from txai_omics_3.models.tabular.widedeep.ft_transformer import WDFTTransformerModel, FN_CHECKPOINT, FN_SHAP, TRAIN_DATA_PATH
from llamba_library.bioage_model import BioAgeModel

# Model
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=fn_model)
bioage_model = BioAgeModel(model=model)

# SHAP
fn_shap = FN_SHAP

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
with open(fn_shap, 'rb') as handle:
    shap_dict = pickle.load(handle)

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint H:\Lobachevsky\txai_omics_3\data\immuno\model.ckpt`
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


### Load data

In [4]:
import pandas as pd

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

my_df = pd.DataFrame(my_data, index=[0])

# Model data

train_data = pd.read_excel(TRAIN_DATA_PATH)

### Initialize a connector

In [5]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)

### Run an analysis task

In [6]:
res = connector.analyze(data=my_df, shap_dict=shap_dict, top_n=3)
print(res['analysis'])

Your bioage is 79 and your aging acceleration is -11, which means you are ageing slower than normal.

Here is some more information about your data. 

PDGFA: 1005.84429
CD40LG: 1853.847406
CXCL9: 2599.629474
PDGFA stands for Platelet-Derived Growth Factor A. It's a protein involved in cell signaling pathways that regulate cellular processes such as proliferation, migration, and differentiation.

An increased level of PDGFA has been linked to various age-related diseases, including:

* Cancer: High PDGFA levels have been observed in several types of cancer, particularly those affecting older adults, such as breast, lung, and colon cancers.
* Atherosclerosis: Elevated PDGFA levels can contribute to the development and progression of atherosclerosis, which is characterized by the buildup of plaque in blood vessel walls.
* Neurodegenerative disorders: Increased PDGFA expression has been reported in conditions like Alzheimer's disease, Parkinson's disease, and amyotrophic lateral sclerosis 